In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
import art

art.TrainableModel(
    name="blue-fly",
    project="my-project",
    base_model="Qwen/Qwen2.5-14B-Instruct",
    _internal_config=art.dev.InternalModelConfig(
        init_args={"enable_prefix_caching": True}
    )
)._internal_config

{'init_args': {'enable_prefix_caching': True}}